### 조선 일보 데이터크롤러

```
Arguments : query_word, (period)

return : title_list, category_list, body_list
```

In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
def body_extractor(link):
    soup = BeautifulSoup(requests.get(link).content, 'html.parser')
    
    text = ""
    for t in soup.select(".par"):
        text += " " + t.text

    return text.strip()

def chosun_crawler(query, period=None):
    
    # data containers 
    titles = []
    links = []
    categories = []
    dates = []
    bodies = []
    
    i = 1 
    nobody = 0
    
    while True:
        print("-" * 30)
        print('{} page is start'.format(i))
        
        url = "http://search.chosun.com/search/news.search?query={}&pageno={}&orderby=news".format(query, i)
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        
        if soup.select(".result_none"):
            print('***** no result! *****')
            break

        for art, info in zip(soup.select("dt > a"), soup.select(".art_info")):
            
            title = art.text
            if len(title) == 0:
                nobody += 1
                continue
                
            link = art["href"]
            if len(link) == 0:
                nobody += 1
                continue
                
            category = info.select_one(".bread_crumbs").text
            if len(category) == 0:
                category = 'unknown_category'
                
            date = info.select_one(".date").text
            if len(date) == 0:
                date = "unknown_date"
            
            try:
                body = body_extractor(link)
                if len(body) == 0:
                    print("no body")
                    nobody += 1
                    continue
                    
            except Exception as ex:
                print(ex)
                nobody += 1
                continue
                
            titles.append(title)
            links.append(link)
            categories.append(category) 
            dates.append(date)
            bodies.append(body)
            
        print('{} page is done'.format(i))
        i += 1
        break
    return [titles, links, categories, dates, bodies], nobody

In [7]:
data_dust, nobody_dust = chosun_crawler('미세먼지')

------------------------------
1 page is start
1 page is done


##### 1st 시도
-  page : 253
-  nobody_dust : 59

##### 2nd 시도

- page : 861
- Memory error

In [8]:
import pandas as pd

df = pd.DataFrame()

df["title"] = data_dust[0]
df["link"] = data_dust[1]
df["category"] = data_dust[2]
df["date"] = data_dust[3]
df["body"] = data_dust[4]
df

,title,link,category,date,body
0,"세종시 지역구로 둔 이해찬, 환경장관에 ""세종洑 철거, 시간 두고 판단했으면""",http://news.chosun.com/site/data/html_dir/2019...,조선닷컴 > 정치,2019. 6. 7 (금),"환경부, 2월 세종보 철거 결정…농민 등 주민 반발 ""경관 악화됐다"" 비판도與소속 ..."
1,"출근길 비소식...""우산 챙기세요""",http://news.chosun.com/site/data/html_dir/2019...,조선닷컴 > 사회,2019. 6. 7 (금),금요일인 7일은 남부지방을 지나는 저기압의 영향으로 전국이 흐리고 비가 내리다가 오...
2,역세권·학세권 갖추고 천안 원도심 재생사업 프리미엄 누린다,http://news.chosun.com/site/data/html_dir/2019...,조선일보 > 특집섹션,2019. 6. 7 (금),최근 정부가 노후화된 원도심을 중심으로 도시재생사업을 추진하면서 낙후된 도심의 개발...
3,"""하필 폭풍우 오는 날""..'컬투쇼' 산들X우주소녀, 흐린 날씨에도 쾌청한 라이브[종합]",http://news.chosun.com/site/data/html_dir/2019...,OSEN > 연예,2019. 6. 6 (목),"[OSEN=심언경 기자] B1A4 산들, 우주소녀가 흐린 날씨와 상반되는, 밝은 입..."
4,"NPB 구단들,KT팬서비스 벤치마킹 위해 방문",http://news.chosun.com/site/data/html_dir/2019...,OSEN > 스포츠,2019. 6. 6 (목),[OSEN=조형래 기자] 일본 프로야구 구단들이 IT(정보기술) 시스템과 마케팅 벤...
5,현충일 비 소식… 제주서 시작해 밤부터 전국으로 확대,http://news.chosun.com/site/data/html_dir/2019...,연합뉴스 > 사회,2019. 6. 6 (목),현충일이자 목요일인 6일은 전국에 가끔 구름 많다가 낮부터 흐려지며 제주도를 시작으...
6,현충일 오후 제주 시작으로 7일까지 전국에 비…'돌풍 주의',http://news.chosun.com/site/data/html_dir/2019...,조선닷컴 > 사회,2019. 6. 5 (수),현충일인 6일 오후 들어 제주를 시작으로 오는 7일까지 전국에 비가 오겠다. 비소식...
7,"한국암웨이, 코엑스에서 한국암웨이 미래재단 설립행사 가져",http://biz.chosun.com/site/data/html_dir/2019/...,조선경제i > 피플,2019. 6. 5 (수),한국암웨이(대표이사 김장환)는 지난 4일 인터컨티넨탈 서울 코엑스에서 ‘한국암웨이 ...
8,"文대통령, 반년새 7번 PK행...이번엔 김경수 지사도 만나",http://news.chosun.com/site/data/html_dir/2019...,조선닷컴 > 정치,2019. 6. 5 (수),"창원 수소 시내버스 개통식 참석…수소차량 세번째 시승환경의날 기념식 참석해 축사…""..."
9,"르노삼성, 전기차 시대 이끄는 AS 리더십 주목",http://car.chosun.com/site/data/html_dir/2019/...,카라이프 > 뉴스,2019. 6. 5 (수),전기차 등록 대수는 2017년 2만5000여대에서 지난해 5만5000여대로 2배 이...


In [27]:
df.to_csv("chosun_dust.csv", index=False)

In [33]:
import pickle 

with open('chosun_dust.pkl', "wb") as f:
    pickle.dump(df, f)

In [34]:
with open('chosun_dust.pkl', "rb") as f:
    sla = pickle.load(f)

In [35]:
sla

,title,link,category,date,body
0,"세종시 지역구로 둔 이해찬, 환경장관에 ""세종洑 철거, 시간 두고 판단했으면""",http://news.chosun.com/site/data/html_dir/2019...,조선닷컴 > 정치,2019. 6. 7 (금),"환경부, 2월 세종보 철거 결정…농민 등 주민 반발 ""경관 악화됐다"" 비판도與소속 ..."
1,"출근길 비소식...""우산 챙기세요""",http://news.chosun.com/site/data/html_dir/2019...,조선닷컴 > 사회,2019. 6. 7 (금),금요일인 7일은 남부지방을 지나는 저기압의 영향으로 전국이 흐리고 비가 내리다가 오...
2,역세권·학세권 갖추고 천안 원도심 재생사업 프리미엄 누린다,http://news.chosun.com/site/data/html_dir/2019...,조선일보 > 특집섹션,2019. 6. 7 (금),최근 정부가 노후화된 원도심을 중심으로 도시재생사업을 추진하면서 낙후된 도심의 개발...
3,"""하필 폭풍우 오는 날""..'컬투쇼' 산들X우주소녀, 흐린 날씨에도 쾌청한 라이브[종합]",http://news.chosun.com/site/data/html_dir/2019...,OSEN > 연예,2019. 6. 6 (목),"[OSEN=심언경 기자] B1A4 산들, 우주소녀가 흐린 날씨와 상반되는, 밝은 입..."
4,"NPB 구단들,KT팬서비스 벤치마킹 위해 방문",http://news.chosun.com/site/data/html_dir/2019...,OSEN > 스포츠,2019. 6. 6 (목),[OSEN=조형래 기자] 일본 프로야구 구단들이 IT(정보기술) 시스템과 마케팅 벤...
5,현충일 비 소식… 제주서 시작해 밤부터 전국으로 확대,http://news.chosun.com/site/data/html_dir/2019...,연합뉴스 > 사회,2019. 6. 6 (목),현충일이자 목요일인 6일은 전국에 가끔 구름 많다가 낮부터 흐려지며 제주도를 시작으...
6,현충일 오후 제주 시작으로 7일까지 전국에 비…'돌풍 주의',http://news.chosun.com/site/data/html_dir/2019...,조선닷컴 > 사회,2019. 6. 5 (수),현충일인 6일 오후 들어 제주를 시작으로 오는 7일까지 전국에 비가 오겠다. 비소식...
7,"한국암웨이, 코엑스에서 한국암웨이 미래재단 설립행사 가져",http://biz.chosun.com/site/data/html_dir/2019/...,조선경제i > 피플,2019. 6. 5 (수),한국암웨이(대표이사 김장환)는 지난 4일 인터컨티넨탈 서울 코엑스에서 ‘한국암웨이 ...
8,"文대통령, 반년새 7번 PK행...이번엔 김경수 지사도 만나",http://news.chosun.com/site/data/html_dir/2019...,조선닷컴 > 정치,2019. 6. 5 (수),"창원 수소 시내버스 개통식 참석…수소차량 세번째 시승환경의날 기념식 참석해 축사…""..."
9,"르노삼성, 전기차 시대 이끄는 AS 리더십 주목",http://car.chosun.com/site/data/html_dir/2019/...,카라이프 > 뉴스,2019. 6. 5 (수),전기차 등록 대수는 2017년 2만5000여대에서 지난해 5만5000여대로 2배 이...
